In [25]:
import tensorflow as tf

In [26]:
rank_three_tensor = tf.ones([3,4,5])
print(rank_three_tensor.shape)
matrix = tf.reshape(rank_three_tensor, [3,-1])
print(matrix.shape)

(3, 4, 5)
(3, 20)


In [38]:
node1 = tf.constant(3.4, dtype=tf.float32)
node2 = tf.constant(4.0)
print(node1, node2)

Tensor("Const_17:0", shape=(), dtype=float32) Tensor("Const_18:0", shape=(), dtype=float32)


In [40]:
sess.run([node1, node2])
node3 = tf.add(node1,node2)
print(node3)
sess.run(node3)

Tensor("Add_4:0", shape=(), dtype=float32)


7.4000001

In [63]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-0.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

init = tf.global_variables_initializer()
sess.run(init)

y = tf.placeholder(tf.float32)
squared_delta = tf.square(linear_model - y)
print(sess.run(squared_delta, {x:[1,2,3,4], y: [0, -1, -2, -3]}))
loss = tf.reduce_sum(squared_delta)
print(sess.run(loss,  {x:[1,2,3,4], y: [0, -1, -2, -3]}))

[  0.           1.68999982   6.75999928  15.21000099]
23.66


In [72]:
## Run linear regression

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
train_x = [1, 2, 3, 4]
train_y = [0, -1 ,-2 ,-3]

# reset global variables
sess.run(init)

# train
for i in range(0,1000):
    sess.run(train, {x:train_x, y:train_y})
    
curr_W, curr_b, curr_x = sess.run([W, b, loss], {x: train_x, y : train_y})
print("W: %s, b: %s, loss: %s"%(curr_W, curr_b, curr_x))


W: [-0.9999969], b: [ 0.99999082], loss: 5.69997e-11


In [73]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
show_graph(tf.get_default_graph().as_graph_def())